In [ ]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing import image,sequence

Using TensorFlow backend.


In [ ]:
data = pd.read_csv('./traindat.txt', delimiter='\t',header=None)
print(data.shape)
data.head()


(30000, 2)


,0,1
0,2513260012_03d33305cf.jpg,<start> A black dog is running after a white d...
1,2513260012_03d33305cf.jpg,<start> Black dog chasing brown dog through sn...
2,2513260012_03d33305cf.jpg,<start> Two dogs chase each other across the s...
3,2513260012_03d33305cf.jpg,<start> Two dogs play together in the snow . <...
4,2513260012_03d33305cf.jpg,<start> Two dogs running through a low lying b...


In [ ]:
sentences = []
for ix in range(data.shape[0]):
    sentences.append(data.loc[ix][1])
print(len(sentences))
# print(sentences)

30000


In [ ]:
words = [i.split() for i in sentences]
unique = []
max_len = 0 
for i in words:
    unique.extend(i)
    if(len(i)>max_len):
        max_len = len(i)

unique = list(set(unique))
print(len(unique))
print(max_len)
vocab_size = len(unique)

8253
40


In [ ]:
#Vectorization
word_2_indices = { val:index for index, val in enumerate(unique)}
indices_2_word = { index:val for index, val in enumerate(unique)}
temp = indices_2_word[0]
word_2_indices[temp]= vocab_size
word_2_indices['EMP'] = 0
indices_2_word[0] = 'EMP'
indices_2_word[vocab_size]= temp
vocab_size = len(word_2_indices)
print(vocab_size)

with open('./word_2_indices.p','wb') as f1:
    pickle.dump(word_2_indices,f1)
with open('./indices_2_words.p','wb') as f2:
    pickle.dump(indices_2_word,f2)
f1.close()
f2.close()

8254


In [ ]:
padded_sequences, subsequent_words = [], []

for ix in range(len(sentences)):
    partial_seqs = []
    next_words = []
    text = sentences[ix].split()
    text = [word_2_indices[i] for i in text]
    for i in range(1, len(text)):
        partial_seqs.append(text[:i])
        next_words.append(text[i])
    
    padded_partial_seqs = sequence.pad_sequences(partial_seqs, max_len, padding='post')
    next_words_1hot = np.zeros([len(next_words), vocab_size], dtype=np.bool)
    
    #Vectorization
    for i,next_word in enumerate(next_words):
        next_words_1hot[i, next_word] = 1
        
    padded_sequences.append(padded_partial_seqs)
    subsequent_words.append(next_words_1hot)
    
padded_sequences = np.asarray(padded_sequences)
subsequent_words = np.asarray(subsequent_words)

print(padded_sequences.shape)
print(subsequent_words.shape)

(30000,)
(30000,)


In [ ]:
print(padded_sequences[0].shape)
print(subsequent_words[0].shape)

(14, 40)
(14, 8254)


In [ ]:
for ix in range(len(padded_sequences[0])):
    for iy in range(max_len):
        print(indices_2_word[padded_sequences[0][ix][iy]],end=' ')
    print("\n")

print(len(padded_sequences[0]))

<start> EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP 

<start> A EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP 

<start> A black EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP 

<start> A black dog EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP 

<start> A black dog is EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP 

<start> A black dog is running EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP EMP 

<sta

In [ ]:
with open('./train_encoded_images.p', 'rb') as f:
        encoded_images = pickle.load(f)

imgs = []
for ix in range(data.shape[0]):
    if data.loc[ix, 0] in encoded_images.keys():
        imgs.append(list(encoded_images[data.loc[ix, 0]]))

    else:
        imgs.append(list([]))
images = []
image_names = []

imgs = np.asarray(imgs)
print(imgs.shape)

captions = np.zeros([0, max_len])
print(captions.shape)
next_words = np.zeros([0, vocab_size])
print(next_words.shape)
for ix in range(2000):
    if(len(imgs[ix])!=0):
        captions = np.concatenate([captions, padded_sequences[ix]])
        next_words = np.concatenate([next_words, subsequent_words[ix]])
        for iy in range(padded_sequences[ix].shape[0]):
            image_names.append(data.loc[ix][0])
            images.append(imgs[ix])
        
    if(ix%100==0):
        print(ix,end=' ')
image_names = np.asarray(image_names)
images = np.asarray(images)
        
np.save("./captions.npy", captions)
np.save("./next_words.npy", next_words)
np.save("./image_names.npy", image_names)
np.save("./images.npy", images)


(30000,)
(0, 40)
(0, 8254)
0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 

In [ ]:
print(captions.shape)
print(next_words.shape)
print(len(image_names))
print(len(images))

(25493, 40)
(25493, 8254)
25493
25493
